In [1]:
import sys, math
from collections import namedtuple
import numpy as np
import pandas as pd
import pygame
import random
import lzma
import os

DATA_DIRECTORY = "data/2023_2/"
INPUT_FILE = 'KA050_processed_10cm_5h_20230614.pkl.xz'

def load_data(source_dir, input_file, scale = None, arena_dim = None):
    data = None
    with lzma.open(os.path.join(source_dir, input_file)) as file:
        data = pd.read_pickle(file)
    return data.iloc[::int(scale)] if scale else data


def process_data(data, arena_dim):
    data_len = len(data)
    arena_bb = find_bounding_box(data)
    origin_arena = calculate_circle(*arena_bb)

    translation, scale = circle_transformation(origin_arena, arena_dim)

    apply_transform_scale(data, translation, scale)

    return data

data = load_data(DATA_DIRECTORY, INPUT_FILE)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
data.head()

0             1             2             3             4          ...  \
       x      y      x      y      x      y      x      y      x      y  ...   
0  180.0  225.0  339.0  591.0  326.0  614.0  308.0  750.0  294.0  158.0  ...   
1  180.0  225.0  340.0  592.0  325.0  614.0  308.0  750.0  294.0  158.0  ...   
2  180.0  225.0  340.0  592.0  325.0  614.0  308.0  749.0  294.0  158.0  ...   
3  180.0  224.0  340.0  592.0  324.0  614.0  308.0  749.0  294.0  158.0  ...   
4  180.0  224.0  340.0  592.0  324.0  614.0  308.0  749.0  294.0  158.0  ...   

   52      53      54      55      56      
    x   y   x   y   x   y   x   y   x   y  
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[5 rows x 114 columns]

In [3]:
data[0].head()

,x,y
0,180.0,225.0
1,180.0,225.0
2,180.0,225.0
3,180.0,224.0
4,180.0,224.0


## Step 1: Data Preprocessing
### A. Data Cleaning**
**1. Handling Missing Data**
This dataset contains NaN values, which likely indicate missing positional data for certain entities at specific time steps. We'll need to handle these appropriately.

Approach:

Do not interpolate missing values. NaN values indicate that the ant was no longer being tracked, and a separate track was generated. Please retain all NaN values in their current positions for now.

**2. Smoothing Trajectories**
Positional data can be noisy due to tracking inaccuracies. Smoothing helps in obtaining cleaner velocity and acceleration estimates.

Approach:

Gaussian Smoothing: Applies a Gaussian filter to reduce noise while preserving essential movement patterns.
Moving Average: Simple method to average out short-term fluctuations.
Implementation:

python
Copy code


In [ ]:
from scipy.ndimage import gaussian_filter1d

# Apply Gaussian smoothing to each 'x' and 'y' column
smoothed_data = data_interpolated.copy()

for col in data_interpolated.columns:
    if col[1] in ['x', 'y']:
        smoothed_data[col] = gaussian_filter1d(data_interpolated[col], sigma=2)

# Alternatively, using moving average
# window_size = 3
# smoothed_data = data_interpolated.rolling(window=window_size, min_periods=1).mean()
